In [35]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor
from sklearn import metrics

In [36]:
df = pd.read_csv('../data/train_data_encoded_full.csv')
print('shape:', df.shape)
df.head(5)

shape: (39829, 7)


,weight,quality,color,clarity,table_percent,price,depth_percent
0,0.30,3,6,1,0.58,1.848927,0.624
1,1.01,4,5,5,0.56,2.217354,0.627
2,0.72,4,4,3,0.59,2.077314,0.618
3,1.08,2,3,1,0.57,2.124773,0.632
4,0.36,3,3,4,0.59,1.885250,0.623


In [37]:
df['weight'] = df['weight'] * 0.2
df

,weight,quality,color,clarity,table_percent,price,depth_percent
0,0.060,3,6,1,0.58,1.848927,0.624
1,0.202,4,5,5,0.56,2.217354,0.627
2,0.144,4,4,3,0.59,2.077314,0.618
3,0.216,2,3,1,0.57,2.124773,0.632
4,0.072,3,3,4,0.59,1.885250,0.623
...,...,...,...,...,...,...,...
39824,0.084,3,6,1,0.59,1.879618,0.621
39825,0.106,3,3,3,0.58,1.999045,0.620
39826,0.160,1,3,1,0.58,2.050013,0.628
39827,0.202,2,4,3,0.57,2.166307,0.615


In [38]:
#split dara train and test
X = df.drop(["price"], axis = 1)
y = df['price']

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size   = 0.8, random_state = 42)

print("Datos de entrenamiento")
print("-----------------------")
print(y_train.describe())
print('\n')
print("Datos de testeo")
print("-----------------------")
print(y_test.describe())

Datos de entrenamiento
-----------------------
count    31863.000000
mean         2.042597
std          0.131438
min          1.755614
25%          1.923080
50%          2.050270
75%          2.149201
max          2.286659
Name: price, dtype: float64


Datos de testeo
-----------------------
count    7966.000000
mean        2.042922
std         0.131004
min         1.762331
25%         1.924687
50%         2.051106
75%         2.148734
max         2.286659
Name: price, dtype: float64


In [39]:
def metricas(y_test, y_train, y_test_pred, y_train_pred, tipo_modelo):
    resultados = {'MAE': [metrics.mean_absolute_error(y_test, y_test_pred), metrics.mean_absolute_error(y_train, y_train_pred)],
                'MSE': [metrics.mean_squared_error(y_test, y_test_pred), metrics.mean_squared_error(y_train, y_train_pred)],
                'RMSE': [np.sqrt(metrics.mean_squared_error(y_test, y_test_pred)), np.sqrt(metrics.mean_squared_error(y_train, y_train_pred))],
                'R2':  [metrics.r2_score(y_test, y_test_pred), metrics.r2_score(y_train, y_train_pred)],
                 "set": ["test", "train"]}
    df = pd.DataFrame(resultados)
    df["modelo"] = tipo_modelo
    return df

In [40]:
gb = GradientBoostingRegressor(n_estimators=100, loss='squared_error', learning_rate=0.1, max_depth=7, max_features=7, random_state=0, criterion='friedman_mse')
gb.fit(X_train, y_train)

y_pred_gb_test= gb.predict(X_test)
y_pred_gb_train= gb.predict(X_train)

results_gradient_1 = metricas(y_test, y_train, y_pred_gb_test, y_pred_gb_train, "Gradient 1")
results_gradient_1.style.background_gradient(cmap='coolwarm')

,MAE,MSE,RMSE,R2,set,modelo
0,0.009366,0.000158,0.012565,0.990800,test,Gradient 1
1,0.008637,0.000133,0.011537,0.992295,train,Gradient 1


In [41]:
gb = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, max_depth=9, max_features=5, random_state=0, criterion='squared_error')
gb.fit(X_train, y_train)

y_pred_gb_test= gb.predict(X_test)
y_pred_gb_train= gb.predict(X_train)

results_gradient_2 = metricas(y_test, y_train, y_pred_gb_test, y_pred_gb_train, "Gradient 2")
results_gradient_2.style.background_gradient(cmap='coolwarm')

,MAE,MSE,RMSE,R2,set,modelo
0,0.009470,0.000164,0.012811,0.990436,test,Gradient 2
1,0.007432,0.000104,0.010187,0.993993,train,Gradient 2


In [42]:
gb = GradientBoostingRegressor(n_estimators=125, loss='squared_error', learning_rate=0.15, max_depth=6, max_features=8, random_state=0, criterion='squared_error')
gb.fit(X_train, y_train)

y_pred_gb_test= gb.predict(X_test)
y_pred_gb_train= gb.predict(X_train)

results_gradient_3 = metricas(y_test, y_train, y_pred_gb_test, y_pred_gb_train, "Gradient 3")
results_gradient_3.style.background_gradient(cmap='coolwarm')

,MAE,MSE,RMSE,R2,set,modelo
0,0.009425,0.000159,0.012611,0.990732,test,Gradient 3
1,0.008798,0.000137,0.011720,0.992049,train,Gradient 3


In [43]:
df_results = pd.concat([results_gradient_1, results_gradient_2, results_gradient_3], axis = 0)
df_results

,MAE,MSE,RMSE,R2,set,modelo
0,0.009366,0.000158,0.012565,0.990800,test,Gradient 1
1,0.008637,0.000133,0.011537,0.992295,train,Gradient 1
0,0.009470,0.000164,0.012811,0.990436,test,Gradient 2
1,0.007432,0.000104,0.010187,0.993993,train,Gradient 2
0,0.009425,0.000159,0.012611,0.990732,test,Gradient 3
1,0.008798,0.000137,0.011720,0.992049,train,Gradient 3
